In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
from datetime import datetime

#configuracion
opciones = Options()
opciones.add_argument("--headless")
opciones.add_argument("--disable-gpu")
opciones.add_argument("--window-size=1920,1080")
opciones.add_argument("--no-sandbox")
opciones.add_argument("--disable-dev-shm-usage")
opciones.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")


#variables principales
datos_trabajos = []
fecha_hora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
archivo_salida = 'trabajos_bumeran.xlsx'
total_registros = 0
guardar_cada = 200 
max_paginas = 1530

def guardar_excel():
    global datos_trabajos
    
    if not datos_trabajos:
        return
        
    df = pd.DataFrame(datos_trabajos)

    if os.path.exists(archivo_salida):
        df_existente = pd.read_excel(archivo_salida)
        df_final = pd.concat([df_existente, df], ignore_index=True)
        df_final.to_excel(archivo_salida, index=False)
    else:
        df.to_excel(archivo_salida, index=False)
    
    print(f"Guardado: {total_registros} trabajos")
    datos_trabajos.clear()

try:

    print("Chrome")
    navegador = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=opciones
    )

    for pagina in range(1, max_paginas + 1):
        try:
            url = f"https://www.bumeran.com.pe/empleos.html?page={pagina}"
            print(f"Página {pagina}/{max_paginas}")
            navegador.get(url)

            espera = WebDriverWait(navegador, 20)
            espera.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[class*='sc-eUNjYF'][class*='sc-jrOYZv']")))

            sopa = BeautifulSoup(navegador.page_source, 'html.parser')
            lista_trabajos = sopa.find_all("div", class_=lambda c: c and "sc-eUNjYF" in c and "sc-jrOYZv" in c)
            
            if not lista_trabajos:
                print(f"Sin resultados, terminando.")
                break
                
            # procesar trabajos
            for trabajo in lista_trabajos:
                try:
                    # defaults
                    nombre_puesto = ""
                    nombre_empresa = ""
                    ubicacion = ""
                    tipo_contrato = ""
                    descripcion = ""
                    link = ""
                    fecha_publicacion = ""

                    elem_link = trabajo.find("a", class_=lambda c: c and "sc-jrOYZv" in c)
                    if elem_link:
                        link = "https://www.bumeran.com.pe" + elem_link.get("href")
                    
                    elem_puesto = trabajo.find("h2")
                    if elem_puesto:
                        nombre_puesto = elem_puesto.text.strip()

                    elem_empresa = trabajo.find("span", class_=lambda c: c and "sc-bvODop" in c)
                    if elem_empresa and elem_empresa.find("h3"):
                        nombre_empresa = elem_empresa.find("h3").text.strip()
                    

                    for elem in trabajo.find_all("h3"):
                        if "Publicado" in elem.text or "Actualizado" in elem.text:
                            fecha_publicacion = elem.text.strip()
                            break
                    
                    if not fecha_publicacion:
                        elem_actualizado = trabajo.find("h3", class_=lambda c: c and "sc-eNGGxe" in c)
                        if elem_actualizado and ("Actualizado" in elem_actualizado.text):
                            fecha_publicacion = elem_actualizado.text.strip()

                    for elem in trabajo.find_all("div", class_=lambda c: c and "sc-fjNYmT" in c):
                        if elem.find("i", {"name": "icon-light-location-pin"}) and elem.find("h3"):
                            ubicacion = elem.find("h3").text.strip()
                        elif elem.find("i", {"name": "icon-light-office"}) and elem.find("h3"):
                            tipo_contrato = elem.find("h3").text.strip()
                    
                    elem_descripcion = trabajo.find("p")
                    if elem_descripcion:
                        descripcion = elem_descripcion.text.strip()
                        descripcion = descripcion[:500] + '...' if len(descripcion) > 500 else descripcion
                        
                    # guardar datos
                    datos_trabajos.append({
                        'Puesto': nombre_puesto,
                        'Empresa': nombre_empresa,
                        'Ubicacion': ubicacion,
                        'TipoContrato': tipo_contrato,
                        'Descripcion': descripcion,
                        'Enlace': link,
                        'FechaPublicacion': fecha_publicacion,
                        'FechaScraping': fecha_hora,
                        'NumPagina': pagina
                    })
                    
                    total_registros += 1
                    
                except Exception as e:
                    print(f"Error trabajo: {e}")
            
            # guardado periódico
            if total_registros % guardar_cada == 0:
                guardar_excel()
            


            print(f"Procesados: {total_registros}")
            time.sleep(2)
            
        except Exception as e:
            print(f"Error página {pagina}: {e}")
            continue
    
    # guardar restantes
    if datos_trabajos:
        guardar_excel()
    
except Exception as e:
    print(f"Error: {e}")

finally:

    if datos_trabajos:
        guardar_excel()
    
    try:
        navegador.quit()
    except:
        pass
    
    print(f"Terminado Total: {total_registros}")